## Authlib

Authlib is a package that profides intefaces for handling authentication through OAuth and OpenID. It also provides specific integrations with popular web frameworks.

## Removed session

Consider what happens when a session is removed and the application attempts to refresh the token. Typicallyk, OAuth returns the 400 error.

---

The following cell requests the token that creates a new Keycloak sesion.

In [35]:
from authlib.integrations.requests_client import OAuth2Session
client = OAuth2Session(client_id="admin-cli")
token = client.fetch_token(
    "http://localhost:8080/realms/master/protocol/openid-connect/token",
    username="admin",
    password="admin",
    grant_type="password",
)

The text code removes the session, emitating that session was accidentally dropped.

In [36]:
access_token = token["access_token"]
session_state = token["session_state"]

!curl -X DELETE  \
     -H "Authorization: Bearer $access_token" \
     "http://localhost:8080/admin/realms/master/sessions/$session_state"

Requesting a token refresh results in `OAuthError` with the relevant description.

In [39]:
try:
    client.fetch_token(
        "http://localhost:8080/realms/master/protocol/openid-connect/token",
        grant_type="refresh_token",
        refresh_token=token["refresh_token"]
    )
except Exception as e:
    print(type(e))
    print(e)

<class 'authlib.integrations.base_client.errors.OAuthError'>
invalid_grant: Session not active


## Requests integration

The `authlib` provides the `authlib.integrations.requests_client.OAuth2Session` class - the custom `requests.Session` to which the authentication logic is included.

---

The following cell shows the `mro` of `authlib.integrations.requests_client`.

In [172]:
from authlib.integrations.requests_client import OAuth2Session
OAuth2Session.__mro__

(authlib.integrations.requests_client.oauth2_session.OAuth2Session,
 authlib.oauth2.client.OAuth2Client,
 requests.sessions.Session,
 requests.sessions.SessionRedirectMixin,
 object)

The list of ancestors includes `requests.sessions.Session`.

This allows to use the `OAuth2Session` instance as a regular instance of `requests.sessions.Session` and not to care about the injecting the authentication information to the header.

The following code uses the `get` method of the `OAuth2Session`: 

In [173]:
client = OAuth2Session(client_id="admin-cli", leeway=30)
client.fetch_token(
    "http://localhost:8080/realms/master/protocol/openid-connect/token",
    username="admin",
    password="admin",
    grant_type="password",
)
response = client.get("http://localhost:8080/admin/realms/master/users/count")
response.status_code, response.text

(200, '1')